In [56]:
import QuantLib as ql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [57]:
def p():   # pricing function
    sigma = 0.02   # HW model sigma parameter
    a = 0.04   # HW model a parameter
    length = 3   # Simulation time (in years)
    steps_per_year = 252   # daily basis
    steps = length * steps_per_year
    forward_rate = 0.0011   # forward rate
    count = ql.Thirty360()
    date = ql.Date(9, 5, 2022)

    ql.Settings.instance().evaluationDate = date

    quote_Handle = ql.QuoteHandle(ql.SimpleQuote(forward_rate))
    spot_curve = ql.FlatForward(date, quote_Handle, count)
    spot_curve_handle = ql.YieldTermStructureHandle(spot_curve)

    hw_process = ql.HullWhiteProcess(spot_curve_handle, a, sigma)
    rdm_gen = ql.UniformRandomGenerator()
    rdm_seq = ql.UniformRandomSequenceGenerator(steps, rdm_gen)
    rng = ql.GaussianRandomSequenceGenerator(rdm_seq)
    seq = ql.GaussianPathGenerator(hw_process, length, steps, rng, False)

    def paths(N):
        arr = np.zeros((N, steps + 1))
        for i in range(N):
            sample = seq.next()
            path = sample.value()
            time = [path.time(j) for j in range(len(path))]
            value = [path[j] for j in range(len(path))]
            arr[i, :] = np.array(value)
        return np.array(time), arr

    N = 1000   # number of simulations
    time, paths = paths(N)
    rates = pd.DataFrame(paths).T

    n = length
    num_steps = steps
    dt = 1 / steps_per_year
    prices = np.empty_like(rates)

    def price(T, t, f0, rt):   # pricing based on HW model
        tau = T - t
        B = (1 - np.exp(-a * tau)) / a
        A = np.exp(-f0 * tau + B * f0 - sigma ** 2 / (4 * a ** 3) *
                    (np.exp(-a * T) - np.exp(-a * t)) * (np.exp(2 * a * t) - 1))
        return A * np.exp(-rt * B)

    pr = price(n, n - 90 * dt, forward_rate, rates.values[num_steps - 90])
    ave = np.mean(pr)

    L_d = -(pr - 1) / (pr)   # pricing Libor equity
    p0 = price(n, 0, forward_rate, rates.values[0])
    p_d = price(n - 90 * dt, 0, forward_rate, rates.values[0])
    L_0 = -(p0 - p_d) / (p0 * 90 * dt)   # pricing Libor equity


    def geo_paths(S, T, r, q, sigma_s, sigma_x, rho, steps, N):
         dt = T / steps
         ST = np.log(S) + np.cumsum(((r - q - sigma_s * sigma_x * rho - sigma_s ** 2 / 2) * dt + \
                                      sigma_s * np.sqrt(dt) * \
                                      np.random.normal(size=(steps, N))), axis=0)

         return np.exp(ST)

    S = 27003.560547  # stock price S_{0}
    T = 3  # time to maturity (in year)
    r = 0.02   # risk free risk in annual %
    q = 0.0152  # annual dividend rate
    sigma_S = 0.11966278630595297  # annual volatility in % (stock)
    sigma_X = 0.04055125840025642  # annual volatility in % (exchange rate)
    cor =  0.036397916482133884  # correlation between stock price and exchange rate

    paths = geo_paths(S, T, r, q, sigma_S, sigma_X, cor, steps, N)

    S_T = paths[-1]   # pricing quanto equity

    k = 1  # strike
    
    Pi = (k - L_d / L_0) * (S_T / S - k)
    for i in range(1000):
        if Pi[i] < 0:
            Pi[i] = 0   #Take max(0, Pi)

    mean = np.mean(Pi)
    return mean


In [ ]:
P_s = []
for i in range(200):
    P_s.append(p())

In [ ]:
print("Mean of the simulated contract price", np.mean(P_s))

In [ ]:
print("90% of the simulated contract price interval", np.percentile(P_s, [5, 95]))

In [ ]:
plt.hist(P_s, density=True, bins=10)  # density=False would make counts
plt.title("200 Times Pricing Histogram")
plt.ylabel('Probability')
plt.xlabel('Contract price');
plt.savefig("HW_histogram.pdf")